In [1]:
#Clean notebook for data clean up explantions

In [2]:
#imported our Dependancies as needed
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import requests
import json
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

#Importing CSV files found online
state_lockdown_file = pd.read_csv("data_sources/lockdown_us.csv")
state_deaths_cases_file = pd.read_csv("data_sources/liz-friedman-us-covid-19-data-from-nytimes/us-states.csv")
county_population_file = pd.read_csv("data_sources/qventus-covid-19-localized-scenario-planner/qventus-covid-19-localized-scenario-planner/covid_county_population_usafacts.csv")

In [3]:
#URL pulls state data just for the current or last weekday reported
url = "https://api.covidtracking.com/v1/states/current.json"
state_current_data = (requests.get(url)).json()
#Creating dataframe form api request
state_current_data=pd.DataFrame(state_current_data)
#date format is yyyymmdd this line changes to yyyy-mm-dd
state_current_data['date'] = pd.to_datetime(state_current_data['date'], format='%Y%m%d')

#Dropping US territoires from state current data
state_current_data = state_current_data.set_index("state")
state_current_data = state_current_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_current_data = state_current_data.reset_index()

state_current_data.head(50)

,state,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,AK,2020-09-02,6233,370651,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,
1,AL,2020-09-02,128239,835994,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,
2,AR,2020-09-02,62112,675337,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,
3,AZ,2020-09-02,202861,1011066,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,
4,CA,2020-09-02,712052,10868318,NaN,4851.0,NaN,1339.0,NaN,NaN,...,11580370,145,0,61ce2c9982d886c3da71512bb44db09f2bd735a6,0,0,0,0,0,
5,CO,2020-09-02,57775,660623,NaN,236.0,7053.0,NaN,NaN,NaN,...,718398,4,25,6390dedf19353347575ce408692f0fd3283149ef,0,0,0,0,0,
6,CT,2020-09-02,53108,1150871,NaN,65.0,11180.0,NaN,NaN,NaN,...,1203979,1,0,01ab76762b7d3161278f70030767edd0184b0e60,0,0,0,0,0,
7,DE,2020-09-02,17549,222981,NaN,62.0,NaN,9.0,NaN,NaN,...,240530,1,0,add38598290fa882e2083f84bed00763579b58a6,0,0,0,0,0,
8,FL,2020-09-02,633442,4060360,3515.0,3511.0,39611.0,NaN,NaN,NaN,...,4693802,130,300,1d3a0eb59cf90c76cb7f2bd307324bee0e49f4f9,0,0,0,0,0,
9,GA,2020-09-02,274613,2107024,NaN,2469.0,25025.0,NaN,4588.0,NaN,...,2381637,62,178,f0b119d1023c46cc465e24a6f82777f5a0182530,0,0,0,0,0,


In [4]:
#URL pulls collection of all states current data from previous dates
url = "https://api.covidtracking.com/v1/states/daily.json"
state_daily_data = (requests.get(url)).json()

#Creating dataframe form API request
state_daily_data=pd.DataFrame(state_daily_data)

#date format is yyyymmdd this line changes to yyyy-mm-dd
#Used later in adding specific cloumns for "Month" and "Day" for sorting data by timeframe
state_daily_data['date'] = pd.to_datetime(state_daily_data['date'], format='%Y%m%d')

#Dropping US Territories from state list
state_daily_data = state_daily_data.set_index("state")
state_daily_data = state_daily_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_daily_data = state_daily_data.reset_index()

#loop for summing for total state cases
states = state_daily_data["state"].unique()
total_cases_by_state = []

for state in states:
    state_case = state_daily_data.loc[state_daily_data["state"] == state]
    total_cases_by_state.append(state_case["positive"].sum())

state_daily_data.head(500)

,state,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,AK,2020-09-02,6233.0,370651.0,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,
1,AL,2020-09-02,128239.0,835994.0,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,
2,AR,2020-09-02,62112.0,675337.0,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,
3,AZ,2020-09-02,202861.0,1011066.0,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,
4,CA,2020-09-02,712052.0,10868318.0,NaN,4851.0,NaN,1339.0,NaN,NaN,...,11580370,145,0,61ce2c9982d886c3da71512bb44db09f2bd735a6,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,VT,2020-08-24,1566.0,119824.0,NaN,11.0,NaN,NaN,NaN,NaN,...,121390,0,0,9a5c7aa14c4db0e44aa16faed4b43398526e9572,0,0,0,0,0,
496,WA,2020-08-24,71681.0,1308369.0,NaN,373.0,6530.0,NaN,NaN,36.0,...,1380050,6,30,c366ae1927bf49b617def0d4947594a53f889763,0,0,0,0,0,
497,WI,2020-08-24,75619.0,1119622.0,180.0,321.0,5573.0,106.0,991.0,NaN,...,1195241,0,15,566128f9d0be8e1ebfc77eefdc3832657a00ee93,0,0,0,0,0,
498,WV,2020-08-24,9312.0,389342.0,NaN,141.0,NaN,47.0,NaN,23.0,...,398654,1,0,1fb6b7d8b383472d08cfb145c24b75bc95aad3f9,0,0,0,0,0,


In [5]:
#loop for finding positive rate per month
#Positive cases per month / total of positive and negitive test results
months = [1,2,3,4,5,6,7,8]
positive_rates = []
state_list = []
month_list = []

for state in states:
    
    single_state = state_daily_data.loc[state_daily_data["state"] == state]
    single_state['date'] = pd.to_datetime(single_state['date'], format='%Y%m%d')    
    single_state = single_state.sort_values("date")
    single_state['month'] = pd.DatetimeIndex(single_state['date']).month
    single_state['day'] = pd.DatetimeIndex(single_state['date']).day
    for month in months:
        #print(state)
        try:
            single_month = single_state.loc[single_state["month"] == month]
            single_month = single_month.reset_index()
            first_row = single_month.iloc[0]
            last_row = single_month.iloc[-1]
            month_pos = int(last_row["positive"]) - int(first_row["positive"])
            month_tot = int(last_row["posNeg"]) - int(first_row["posNeg"])
            positive_rate = month_pos / month_tot
            positive_rate = "{:.2%}".format(positive_rate)
            positive_rates.append(positive_rate)
            state_list.append(state)
            month_list.append(month)
            
            
        except IndexError:
            #some states had no cases in the first 2 months
            print("no data for this month")
        except ValueError:
            #New Jersey had had values of NaN causing a value error
            print("value issue")
        except ZeroDivisionError:
            #Washington first month total comes out to 0 causing a division by zero error
            print("month total = 0")

no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
value issue
value issue
value issue
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this mont

In [6]:
#Dataframe created from above loop
pos_rates_df = pd.DataFrame({
    "State": state_list,
    "Positive Rate": positive_rates,
    "Month": month_list
})
pos_rates_df.head(25)

,State,Positive Rate,Month
0,AK,3.21%,3
1,AK,1.53%,4
2,AK,0.22%,5
3,AK,0.82%,6
4,AK,2.27%,7
5,AK,2.09%,8
6,AL,13.48%,3
7,AL,7.48%,4
8,AL,8.55%,5
9,AL,10.75%,6


In [7]:
#States total population by summing all counties population
state_pop = []
states = county_population_file["State"].unique()

for state in states:
    state_df = county_population_file.loc[county_population_file["State"] == state]
    state_pop.append(state_df["population"].sum())
    
population_df = pd.DataFrame({
    "state": states,
    "population": state_pop
})

In [8]:
#State deaths cases data imported from sources folder
state_deaths_cases_file.head()

#Sorting alphabetically by state
state_deaths_cases_file = state_deaths_cases_file.rename(columns={"state":"State"})
state_deaths_cases_file = state_deaths_cases_file.sort_values("State")

#Breaking down date into a month and day column for easier filtering by month
state_deaths_cases_file['month'] = pd.DatetimeIndex(state_deaths_cases_file['date']).month
state_deaths_cases_file['day'] = pd.DatetimeIndex(state_deaths_cases_file['date']).day

#Dropping any empty rows from dataframe
state_deaths_cases_file = state_deaths_cases_file.dropna()
state_deaths_cases_file = state_deaths_cases_file.set_index("State")

state_deaths_cases_file.head()

,date,fips,cases,deaths,month,day
State,,,,,,
Alabama,2020-04-24,1,6026,209,4,24
Alabama,2020-07-18,1,65234,1286,7,18
Alabama,2020-05-07,1,9046,369,5,7
Alabama,2020-05-09,1,9668,390,5,9
Alabama,2020-08-18,1,110361,1936,8,18


In [9]:
#Looking at specific state data for states according to lockdown
positive_closings = state_deaths_cases_file.loc[["California","Florida","New York", 
                                                "Kansas","Texas","Washington"],
                                                ["date","cases","month"]]
positive_closings = positive_closings.sort_values("date")
positive_closings.head()

,date,cases,month
State,,,
Washington,2020-01-21,1,1
Washington,2020-01-22,1,1
Washington,2020-01-23,1,1
Washington,2020-01-24,1,1
Washington,2020-01-25,1,1


In [10]:
#Pulling data from specific state prior to the decsion to lock down.
#once data is pulled cases are summed up for each month
california = positive_closings.loc["California"]
cali_closings = california.loc[(california["date"] >= "2020-03-19"),:]
cali_closings = california.loc[(california["date"] <= "2020-05-12"),:]
cali_closings = cali_closings.groupby(by=["month"]).agg(sum)

kansas = positive_closings.loc["Kansas"]
ks_closings = kansas.loc[(kansas["date"]>= "2020-04-19"),:]
ks_closings = kansas.loc[(kansas["date"]<= "2020-05-03"),:]
ks_closings = ks_closings.groupby(by=["month"]).agg(sum)

florida = positive_closings.loc["Florida"]
fl_closings = florida.loc[(florida["date"]>= "2020-04-03"),:]
fl_closings = florida.loc[(florida["date"]<= "2020-04-30"),:]
fl_closings = fl_closings.groupby(by=["month"]).agg(sum)

new_york = positive_closings.loc["New York"]
ny_closings = new_york.loc[(new_york["date"]>= "2020-03-22"),:]
ny_closings = new_york.loc[(new_york["date"]<= "2020-05-15"),:]
ny_closings = ny_closings.groupby(by=["month"]).agg(sum)

washington = positive_closings.loc["Washington"]
wa_closings = washington.loc[(washington["date"]>= "2020-03-23"),:]
wa_closings = washington.loc[(washington["date"]<= "2020-05-04"),:]
wa_closings = wa_closings.groupby(by=["month"]).agg(sum)

texas = positive_closings.loc["Texas"]
tx_closings = texas.loc[(texas["date"]>= "2020-04-02"),:]
tx_closings = texas.loc[(texas["date"]<= "2020-04-30"),:]
tx_closings = tx_closings.groupby(by=["month"]).agg(sum)

In [11]:
#Sorting by states Re-open dates.  Data after the reopen is summed up to show total each month.
cali_reo = california.loc[(california["date"] >= "2020-05-12"),:]
cali_reo = cali_reo.groupby(by=["month"]).agg(sum)

ks_reo = kansas.loc[(kansas["date"]>= "2020-05-03"),:]
ks_reo = ks_reo.groupby(by=["month"]).agg(sum)

fl_reo = florida.loc[(florida["date"]>= "2020-04-30"),:]
fl_reo = fl_reo.groupby(by=["month"]).agg(sum)

ny_reo = new_york.loc[(new_york["date"]>= "2020-05-15"),:]
ny_reo = ny_reo.groupby(by=["month"]).agg(sum)

wa_reo = washington.loc[(washington["date"]>= "2020-05-04"),:]
wa_reo = wa_reo.groupby(by=["month"]).agg(sum)

tx_reo = texas.loc[(texas["date"]>= "2020-04-30"),:]
tx_reo = tx_reo.groupby(by=["month"]).agg(sum)

In [12]:
#Removing some of the counties listed and the types of stay at home/shelter in place


state_lockdown_file['month'] = pd.DatetimeIndex(state_lockdown_file['Date']).month
state_lockdown_file['day'] = pd.DatetimeIndex(state_lockdown_file['Date']).day

state_lockdown_file

,Country,State,County,Date,Type,month,day
0,United States,Alabama,NaN,2020-04-04,Stay at home,4,4
1,United States,Alabama,Birmingham,2020-03-24,Shelter in place,3,24
2,United States,Alaska,NaN,2020-03-28,Stay at home,3,28
3,United States,Arizona,NaN,2020-03-31,Stay at home,3,31
4,United States,California,NaN,2020-03-19,Stay at home,3,19
...,...,...,...,...,...,...,...
168,United States,Virginia,NaN,2020-03-30,Stay at home,3,30
169,United States,Washington,NaN,2020-03-23,Stay at home,3,23
170,United States,West Virginia,NaN,2020-03-24,Stay at home,3,24
171,United States,Wisconsin,NaN,2020-03-25,Stay at home,3,25


In [13]:
#Takeing data from the State current API
state_current_narrow = state_current_data[["state","positive","death","hospitalized"]]
state_current_drop = state_current_narrow.dropna()
state_current_drop
state_current_drop = state_current_drop.set_index('state')
state_current_drop

#Merging with the state population dataframe to compair positive cases and deaths with state total population
state_current_df = pd.merge(state_current_drop, population_df, on='state')

state_current_df.head()

,state,positive,death,hospitalized,population
0,AL,128239,2217,14753.0,4903185
1,AR,62112,841,4341.0,3017804
2,AZ,202861,5065,21449.0,7278717
3,CO,57775,1853,7053.0,5758736
4,CT,53108,4467,11180.0,3565287


In [14]:
#Making a new dataframe of 5 states with early lockdown date
states_early_lock = state_current_drop.loc[["KY","MA","NY","WA","WI"],["death","hospitalized"]]
states_early_lock

#Making dataframe of 5 states with a late lockdown date
states_late_lock = state_current_drop.loc[["FL","GA","KS","SC","AZ"],["death","hospitalized"]]
states_late_lock

,death,hospitalized
state,,
FL,11651,39611.0
GA,5795,25025.0
KS,458,2361.0
SC,2794,7948.0
AZ,5065,21449.0


In [15]:
#Selection of states for having summer tourist attractions
#to see if cases increased during summer months
summer_fun = state_deaths_cases_file.loc[["California","Florida","Hawaii",
                                                "Missouri","Texas","Nevada"],
                                                ["date","cases","month"]]

summer_fun.head()

,date,cases,month
State,,,
California,2020-04-27,45208,4
California,2020-03-18,893,3
California,2020-05-13,73218,5
California,2020-07-23,433175,7
California,2020-05-11,69514,5
